Created By: Anshoo Mehra

### At this stage, we will start creating library called MiniFLow, which is like our own version of Tensorflow .. 

Class Architecture:

--  **Neuron** [ Base Class ] <br>
----  **Input** [ Sub-Class which is only used as Input Layer Node, i.e. performing no operation ]<br>
----  **Add** [ Sub-Class which is only used as Hidden Layer Node. i.e. performing some operation ]<br>

Each of above class has method name foward like Input.forward() or Add.foward() -- these are basically computing values which these are expected to foward, in case of Input it is simply placeholder.



In [115]:
"""
You need to change the Add() class below.
"""

class Neuron:
    def __init__(self, inbound_neurons=[]):
        # Neurons from which this Node receives values
        self.inbound_neurons = inbound_neurons
        # Neurons to which this Node passes values
        self.outbound_neurons = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_neurons:
            n.outbound_neurons.append(self)
        

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_neurons` and
        store the result in self.value.
        """
        raise NotImplemented


class Input(Neuron):
    def __init__(self):
        # an Input neuron has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Neuron.__init__(self)
        print ("*****Instantiated Input Neuron!!")

    # NOTE: Input node is the only node where the value
    # is passed as an argument to forward().
    #
    # All other neuron implementations should get the value
    # of the previous neurons from self.inbound_neurons
    #
    # Example:
    # val0 = self.inbound_neurons[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value
        print ("***Forward Method of Input Instance Called with Value: ", self.value)


class Add(Neuron):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_neurons[0] (`x`) and self.inbound_neurons[1] (`y`)
        Neuron.__init__(self, [x, y])
        print ("*****Instantiated ADD Neuron!!")

    def forward(self):
        """
        Set the value of this neuron (`self.value`) to the sum of it's inbound_nodes.
        
        Your code here!
        """
        x_val= self.inbound_neurons[0].value
        y_val= self.inbound_neurons[1].value
        self.value = x_val+y_val
        #print("xval, yval", x_val, y_val)
        print ("***Forward Method of Add Instance Called with Value: ", self.value)

"""
No need to change anything below here!
"""

'\nNo need to change anything below here!\n'

##### Helper Functions

In [116]:
## Sort generic nodes in topological order using Kahn's Algorithm.
from pprint import pprint
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    print("")
    print("***Inside Topological Sort..")
    ## In this example, it should return pointers to Input Neurons i.e. X & Y
    input_neurons = [n for n in feed_dict.keys()]
    
    ##Custom Data Structure, for ever Node, it holds Input and Output Nodes..
    G = {}
    ## This is just another copy of Input Neurons, Mutable, and will be changed in while loop below to add new
    ## instance of Neurons example outbound infered from inbound..
    neurons = [n for n in input_neurons]

    ## For all Neurons, check then one by one if they exist in G, if not, add them..
    i = 0 
    while len(neurons) > 0:
        i = i+1
        n = neurons.pop(0)
        ## If Instance of Neuron not in G, create with empty placeholder as set() which is unordered, unique by nature. 
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        ## For every instance of Neuron, look for outbound Neurons, remember instance of ADD will set itself as outbound
        ## to X & Y ..
        for m in n.outbound_neurons:
            ## If Instance of Neuron not in G, create with empty placeholder as set() which is unordered, unique by nature. 
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            ## At this stage,
            #### For G entry of Neuron (Input), set outbound Neuron
            G[n]['out'].add(m)
            #### For G entry of Neuron (Output), set inbound Neuron
            G[m]['in'].add(n)
            #### Add Identified Outbound Neuron to the existing list of Neurons, pretty cool as while loop expands
            #### counter & implicitly process newly added nodes (magic of while loop)..
            #### So if you were wondering how ADD Node gets to the list, the magic is sort of started to happen here .. 
            neurons.append(m)

    print("")
    print ("***Cryptic to read, but sort of readable, instance of G with values .. ")
    print("")
    pprint (G)
    print("")
    print("****Did you notice how the list of Neurons (which started with only Input Neurons), now smartly see ADD Node(Neuron) in the list, read comments in code to see how..****")
    print("")
    print("***Time to compute Topological Sort..")
    print("")
    
    ##Placeholder for Sorted Neurons ..
    L = []
    ##Input Neurons ONLY with set() applied to filter non-unique instances if any .. 
    S = set(input_neurons)
    
    ## For every insatnce of Input Neuron..
    while len(S) > 0:
        #Create temporary reference for Input Neuron.. 
        n = S.pop()
        ##Assert if indeed instance is of Class Input, if so, set value from what has been fed in Feed Dictionary ..
        if isinstance(n, Input):
            n.value = feed_dict[n]
        ## Add Instance of Input Neuron to L, this also means it is dependency free and can be
        ## complied / processed trouble free when executed.. 
        L.append(n)
        
        ##Below is hard to understand, since we already added 'n' to 'L'
        ### Remove inbound & outbound entries from G List, you will see why G list, as once G List 'm' ie outbound
        ### node no further inbound nodes len(G[m]['in']) == 0 -- m get added to the S list as last node,
        ### In a way gets sorted to be last node in L for any computation we use this L list for .. 
        for m in n.outbound_neurons:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            #### So if you were wondering how ADD Node gets to the list, the magic is sort of happening here ..
            #### See above comments, how smartly G list is created to help this process step .. 
            if len(G[m]['in']) == 0:
                S.add(m)
    
    print("")
    print("***Topological Sort List..")
    print("")  
    pprint (L)
    print("")
    print("***Completed Topological Sort..")
    print("")
    return L


##Performs a forward pass through a list of sorted neurons.
def forward_pass(output_neuron, sorted_neurons):
    """
    Performs a forward pass through a list of sorted neurons.

    Arguments:

        `output_neuron`: A neuron in the graph, should be the output neuron (have no outgoing edges).
        `sorted_neurons`: a topologically sorted list of neurons.

    Returns the output neuron's value
    """
    print("*****Inside Foward Pass..")
    for n in sorted_neurons:
        n.forward()
    print("*****Foward Pass Completed..")
    print("")
    
    return output_neuron.value

### Let's Implement Neural Nerowrk using above defined library called Miniflow .. 

In [118]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""
# use this import when defined as py files
#from miniflow import *

print ("Before Instantiating Inbound Nodes X & Y")
x, y = Input(), Input()
print("")

print ("Before Instantiating Output Node 'Add' ")
f = Add(x, y)
print("")

feed_dict = {x: 10, y: 5}

sorted_neurons = topological_sort(feed_dict)
output = forward_pass(f, sorted_neurons)

# NOTE: because topological_sort set the values for the `Input` neurons we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

Before Instantiating Inbound Nodes X & Y
*****Instantiated Input Neuron!!
*****Instantiated Input Neuron!!

Before Instantiating Output Node 'Add' 
*****Instantiated ADD Neuron!!


***Inside Topological Sort..

***Cryptic to read, but sort of readable, instance of G with values .. 

{<__main__.Input instance at 0x106087320>: {'in': set([]),
                                            'out': set([<__main__.Add instance at 0x106159950>])},
 <__main__.Add instance at 0x106159950>: {'in': set([<__main__.Input instance at 0x106087320>,
                                                     <__main__.Input instance at 0x106159e60>]),
                                          'out': set([])},
 <__main__.Input instance at 0x106159e60>: {'in': set([]),
                                            'out': set([<__main__.Add instance at 0x106159950>])}}

****Did you notice how the list of Neurons (which started with only Input Neurons), now smartly see ADD Node(Neuron) in the list, read comments in c